In [1]:
import pandas as pd

In [2]:
file_loc = 'out.csv'
df = pd.read_csv(file_loc)

# Utils 

In [3]:
def mask(df,threshold):
    """
    Masks DataFrame values based on threshold, returns new DataFrame with NaN and sign.

    Args:
    df: DataFrame
    threshold: numeric threshold

    Returns:
    DataFrame with masked values.
"""
    df_mask  = df.where(df.abs() > threshold)
    df_mask = df_mask/df_mask.abs()
    return df_mask

# Clean Data

In [4]:
#Add Exchange-contract column
df['Exchange-Contract'] = df['Exchange']+df['Contract']
#Clean columns
df['Basis'] = pd.to_numeric(df['Basis'].str.replace("$","",regex=False))
df['Basis %'] = pd.to_numeric(df['Basis %'].str.replace("%","",regex=False))


# Necessary Dataframes

In [5]:

#Get the percent change of all the Exchange Contract pairs
percent_change_df = df.set_index(['timestamp', 'Exchange-Contract'])['Basis'].unstack(['Exchange-Contract']).pct_change()
percent_change_df = percent_change_df.iloc[1:] #Remove the first row (null)
percent_change_df.head()

Exchange-Contract,BinanceBTCUSDT_230331,BinanceBTCUSD_230331,BinanceBTCUSD_230630,BitMEXXBTF23,BitMEXXBTG23,BitMEXXBTH23,BitMEXXBTM23,BitMEXXBTU23,BitMEXXBTUSDTH23,BitMEXXBTUSDTM23,...,KrakenFI_XBTUSD_230331,KrakenFI_XBTUSD_230630,OKXBTC-USD-230120,OKXBTC-USD-230127,OKXBTC-USD-230331,OKXBTC-USD-230630,OKXBTC-USDT-230120,OKXBTC-USDT-230127,OKXBTC-USDT-230331,OKXBTC-USDT-230630
timestamp,,,,,,,,,,,,,,,,,,,,,
1.674095e+09,0.007198,0.043012,0.023827,0.009074,0.024482,0.116309,-0.575208,0.006643,0.039807,0.005569,...,0.008572,0.003258,-0.115679,-0.022789,-0.017967,-0.028947,0.186081,0.006548,0.057231,-0.001305
1.674095e+09,-0.006187,0.006967,0.003879,0.008993,0.023897,0.020705,-0.094180,0.006600,0.038283,0.005538,...,0.008500,0.003247,0.097715,-0.090825,-0.005934,0.024685,-0.386658,-0.180537,0.007265,0.004501
1.674095e+09,-0.002844,-0.029738,-0.009130,-0.021059,-0.055148,-0.047931,0.246683,-0.015492,-0.087125,-0.013015,...,-0.019914,-0.007648,-0.310840,-0.068512,0.045825,-0.004684,0.278953,0.065167,-0.025185,-0.010588
1.674095e+09,-0.000377,-0.000403,-0.000220,-0.000514,-0.001394,-0.001203,0.004818,-0.000376,-0.002280,-0.000315,...,-0.000485,-0.000184,0.023958,0.113406,-0.000416,-0.011925,0.191339,-0.002174,-0.000418,-0.000256
1.674095e+09,-0.005546,-0.005925,-0.003240,-0.007566,-0.020547,-0.017719,0.069436,-0.005534,-0.033627,-0.004637,...,-0.007146,-0.002710,-0.084435,0.099902,0.014691,0.005264,-0.048249,0.089325,-0.006148,0.003178


In [6]:
#Get the percent change of BTC
#Get one column of the percent change in current_btc_price (all columns have the same value)
btc_price_percent_change = df.set_index(['timestamp', 'Exchange-Contract'])['current_btc_price'].unstack(['Exchange-Contract']).pct_change()['BinanceBTCUSDT_230331']
btc_price_percent_change = btc_price_percent_change.iloc[1:]
btc_price_percent_change.head()

timestamp
1.674095e+09   -0.000481
1.674095e+09    0.000223
1.674095e+09    0.000215
1.674095e+09   -0.000052
1.674095e+09    0.000173
Name: BinanceBTCUSDT_230331, dtype: float64

# Correlation

In [7]:
def calc_correlation(df,price_percent_change):
    percent_change_df = df.copy()
    correlation = percent_change_df.corrwith(btc_price_percent_change)
    return correlation


In [8]:
#Calculate the correlation using default pandas correlation function
correlation = calc_correlation(percent_change_df,btc_price_percent_change)

In [9]:
#Mask the correlation values using a threshold
correlation_thresh = 0.1
masked_correlation = mask(correlation,correlation_thresh)
masked_correlation

Exchange-Contract
BinanceBTCUSDT_230331     NaN
BinanceBTCUSD_230331     -1.0
BinanceBTCUSD_230630     -1.0
BitMEXXBTF23             -1.0
BitMEXXBTG23             -1.0
BitMEXXBTH23              NaN
BitMEXXBTM23              1.0
BitMEXXBTU23             -1.0
BitMEXXBTUSDTH23          NaN
BitMEXXBTUSDTM23         -1.0
BitMEXXBTUSDTU23         -1.0
DeribitBTC-20JAN23        1.0
DeribitBTC-24FEB23        NaN
DeribitBTC-27JAN23        1.0
DeribitBTC-29DEC23       -1.0
DeribitBTC-29SEP23       -1.0
DeribitBTC-30JUN23       -1.0
DeribitBTC-31MAR23        NaN
HuobiDMBTC230120          1.0
HuobiDMBTC230127          1.0
HuobiDMBTC230331          1.0
KrakenFI_XBTUSD_230127   -1.0
KrakenFI_XBTUSD_230331   -1.0
KrakenFI_XBTUSD_230630   -1.0
OKXBTC-USD-230120        -1.0
OKXBTC-USD-230127         NaN
OKXBTC-USD-230331         NaN
OKXBTC-USD-230630         1.0
OKXBTC-USDT-230120        NaN
OKXBTC-USDT-230127        NaN
OKXBTC-USDT-230331        1.0
OKXBTC-USDT-230630        NaN
dtype: float64

In [10]:
#Multiply the mask to the percent_change_df and drop the null columns
masked_percent_change_df = percent_change_df.multiply(masked_correlation).dropna(axis=1)
masked_percent_change_df

Exchange-Contract,BinanceBTCUSD_230331,BinanceBTCUSD_230630,BitMEXXBTF23,BitMEXXBTG23,BitMEXXBTM23,BitMEXXBTU23,BitMEXXBTUSDTM23,BitMEXXBTUSDTU23,DeribitBTC-20JAN23,DeribitBTC-27JAN23,...,DeribitBTC-30JUN23,HuobiDMBTC230120,HuobiDMBTC230127,HuobiDMBTC230331,KrakenFI_XBTUSD_230127,KrakenFI_XBTUSD_230331,KrakenFI_XBTUSD_230630,OKXBTC-USD-230120,OKXBTC-USD-230630,OKXBTC-USDT-230331
timestamp,,,,,,,,,,,,,,,,,,,,,
1.674095e+09,-0.043012,-0.023827,-0.009074,-0.024482,-0.575208,-0.006643,-0.005569,-0.003118,-0.043203,0.386378,...,-0.006242,-0.037622,-0.098416,0.016468,-0.028737,-0.008572,-0.003258,0.115679,-0.028947,0.057231
1.674095e+09,-0.006967,-0.003879,-0.008993,-0.023897,-0.094180,-0.006600,-0.005538,-0.003109,-0.044712,0.055382,...,-0.006203,-0.038710,-0.230836,-0.006614,-0.027934,-0.008500,-0.003247,-0.097715,0.024685,0.007265
1.674095e+09,0.029738,0.009130,0.021059,0.055148,0.246683,0.015492,0.013015,0.007323,0.111048,-0.123995,...,0.014568,0.095542,0.418594,-0.121804,0.064212,0.019914,0.007648,0.310840,-0.004684,-0.025185
1.674095e+09,0.000403,0.000220,0.000514,0.001394,0.004818,0.000376,0.000315,0.000176,0.002433,-0.003382,...,0.000353,0.002123,0.001989,0.074389,0.001639,0.000485,0.000184,-0.023958,-0.011925,-0.000418
1.674095e+09,0.005925,0.003240,0.007566,0.020547,0.069436,0.005534,0.004637,0.002594,0.035157,-0.049927,...,0.005199,0.030685,0.016219,-0.015078,0.024161,0.007146,0.002710,0.084435,0.005264,-0.006148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.674096e+09,0.006872,0.003926,0.013826,0.544643,0.035567,0.008825,0.007002,0.003493,0.049211,-0.095761,...,0.008120,0.079472,0.022612,0.008154,0.022287,0.012677,0.003672,0.099673,-0.002486,-0.004874
1.674096e+09,0.011740,0.006688,0.023788,2.031667,0.058297,0.015107,0.011964,0.005948,0.079611,-0.179688,...,0.013891,0.124963,0.037533,-0.019321,0.038677,0.021785,0.006253,0.187840,-0.007165,-0.020726
1.674096e+09,-0.015151,0.003122,0.011301,-0.909817,0.025495,0.007114,0.005616,0.002775,0.034129,-0.101587,...,0.006533,0.051411,0.016743,0.371838,0.018659,0.010328,0.002918,-0.160894,0.000488,0.009393


# Hit rate 

In [11]:
def calc_hit_rate(df,price_change):
    """
    This function calculates the hit rate of a given DataFrame and a price change. The hit rate is defined as the percentage of times that the price change is greater than 0 for each row of the DataFrame.

    Args:
    df: A pandas DataFrame containing the prices
    price_change: A pandas DataFrame or Series containing the price change for each row of the input DataFrame

    Returns:
    A float representing the hit rate, i.e, the percentage of times that the price change is greater than 0 for each row of the input DataFrame.
    """
    percent_change_df = df.copy()
    hit_rate = percent_change_df.multiply(price_change,axis=0).gt(0).mean()
    return hit_rate

In [12]:
hit_rate = calc_hit_rate(masked_percent_change_df,btc_price_percent_change)
hit_rate

Exchange-Contract
BinanceBTCUSD_230331      0.576471
BinanceBTCUSD_230630      0.670588
BitMEXXBTF23              0.682353
BitMEXXBTG23              0.670588
BitMEXXBTM23              0.670588
BitMEXXBTU23              0.682353
BitMEXXBTUSDTM23          0.682353
BitMEXXBTUSDTU23          0.682353
DeribitBTC-20JAN23        0.658824
DeribitBTC-27JAN23        0.388235
DeribitBTC-29DEC23        0.682353
DeribitBTC-29SEP23        0.682353
DeribitBTC-30JUN23        0.682353
HuobiDMBTC230120          0.670588
HuobiDMBTC230127          0.635294
HuobiDMBTC230331          0.541176
KrakenFI_XBTUSD_230127    0.658824
KrakenFI_XBTUSD_230331    0.682353
KrakenFI_XBTUSD_230630    0.682353
OKXBTC-USD-230120         0.588235
OKXBTC-USD-230630         0.470588
OKXBTC-USDT-230331        0.505882
dtype: float64

# Actual Profit

In [28]:
def calc_actual_profit(df,btc_price_percent_change,actual_profit_thresh=0):
    """
    Calculates the actual profit of a given DataFrame using a given threshold and a btc price change.

    Args:
    df: A pandas DataFrame containing the prices
    actual_profit_thresh: A numeric threshold to mask the values between -thresh and thresh. (default 0)
    btc_price_percent_change: A pandas DataFrame or Series containing the btc price change for each row of the input DataFrame

    Returns:
    A float representing the actual profit by simulating trading by multiplying the mask dataframe and btc price change.
    """
    df_copy = df.copy()
    #Create a dataframe that masks the values between -thresh and thresh. 
    multiplier_df = mask(df_copy,threshold=actual_profit_thresh)

    #Multiply the multiplier_df to the btc_price to simulate trading (same sign = profit, diffrent sign = loss)
    trades_df = multiplier_df.multiply(btc_price_percent_change,axis=0)
    #Get the sum of the trades
    return trades_df
    

In [30]:
trades_df = calc_actual_profit(masked_percent_change_df,btc_price_percent_change)
actual_profit = trades_df.sum()
actual_profit

Exchange-Contract
BinanceBTCUSD_230331      0.002353
BinanceBTCUSD_230630      0.005147
BitMEXXBTF23              0.006427
BitMEXXBTG23              0.006406
BitMEXXBTM23              0.005869
BitMEXXBTU23              0.006427
BitMEXXBTUSDTM23          0.006427
BitMEXXBTUSDTU23          0.006427
DeribitBTC-20JAN23        0.005423
DeribitBTC-27JAN23       -0.003660
DeribitBTC-29DEC23        0.006427
DeribitBTC-29SEP23        0.006427
DeribitBTC-30JUN23        0.006427
HuobiDMBTC230120          0.005863
HuobiDMBTC230127          0.004601
HuobiDMBTC230331          0.002010
KrakenFI_XBTUSD_230127    0.004963
KrakenFI_XBTUSD_230331    0.006427
KrakenFI_XBTUSD_230630    0.006427
OKXBTC-USD-230120         0.003713
OKXBTC-USD-230630         0.000120
OKXBTC-USDT-230331        0.000632
dtype: float64

# Profit Ratio

In [35]:
def calc_profit_ratio(trades_df):
    """
    Calculates the profit ratio of a given DataFrame of trades.
    The profit ratio is calculated as the ratio of the average gain per winning trade to the average loss per losing trade.

    Args:
    trades_df: A pandas DataFrame containing the trades.

    Returns:
    A float representing the profit ratio.
    """

    #calculates the total gain of the trades
    total_gain = trades_df.mul(trades_df.gt(0)).sum()

    #calculates the total loss of the trades
    total_loss = abs(trades_df.mul(~trades_df.gt(0)).sum())

    #calculates the number of winning trades
    num_winning_trades = trades_df.gt(0).sum()

    #calculates the number of losing trades
    num_losing_trades = trades_df.lt(0).sum()
    
    #calculates the profit ratio as the ratio of the average gain per winning trade to the average loss per losing trade
    profit_ratio = (total_gain/num_winning_trades)/(total_loss/num_losing_trades)
    return profit_ratio

In [37]:
profit_ratio = calc_profit_ratio(trades_df)
profit_ratio

Exchange-Contract
BinanceBTCUSD_230331      1.119540
BinanceBTCUSD_230630      1.307853
BitMEXXBTF23              1.683553
BitMEXXBTG23              1.767226
BitMEXXBTM23              1.545543
BitMEXXBTU23              1.683553
BitMEXXBTUSDTM23          1.683553
BitMEXXBTUSDTU23          1.683553
DeribitBTC-20JAN23        1.467260
DeribitBTC-27JAN23        0.806744
DeribitBTC-29DEC23        1.683553
DeribitBTC-29SEP23        1.683553
DeribitBTC-30JUN23        1.683553
HuobiDMBTC230120          1.543015
HuobiDMBTC230127          1.358175
HuobiDMBTC230331          1.213072
KrakenFI_XBTUSD_230127    1.324169
KrakenFI_XBTUSD_230331    1.683553
KrakenFI_XBTUSD_230630    1.683553
OKXBTC-USD-230120         1.381549
OKXBTC-USD-230630         1.148982
OKXBTC-USDT-230331        1.092011
dtype: float64